In [1]:
import json
import pandas as pd
import joblib
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
X = joblib.load('./transformers/X_columns.joblib')

In [3]:
df = pd.DataFrame()
df[X] = None
df['competition_id'] = [0]

In [4]:
df

,competition_id,season,round,home_club_id,away_club_id,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,attendance,referee,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_seats_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_seats_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_num,net_transfer_record_away_num,stadium_code,stadium_name_home_code,stadium_name_away_code
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
with open('./example/example.json', 'r', encoding='utf-8') as f:
    request = json.load(f)

# Формування датасету

In [6]:
competitions = pd.read_csv('../../../data/raw/competitions.csv')

In [7]:
games = pd.read_csv('../../../data/raw/games.csv')

In [8]:
clubs = pd.read_csv('./transformers/clubs_mod.csv')

In [9]:
df['competition_id'] = competitions[competitions['competition_code'] == request['competition_code']]['competition_id'].iloc[0]

In [10]:
df['season'] = pd.to_datetime(request['date']).year

In [11]:
df['round'] = games['round'].mode()[0]

In [12]:
df['home_club_id'] = clubs[clubs['name'] == request['home_club_name']]['club_id'].values
df['away_club_id'] = clubs[clubs['name'] == request['away_club_name']]['club_id'].values

In [13]:
# df['home_club_position'] = games['home_club_position'].mode()[0]
# df['away_club_position'] = games['away_club_position'].mode()[0]
df['home_club_position'] = 3
df['away_club_position'] = 3

In [14]:
df['home_club_manager_name'] = games['home_club_manager_name'].mode()[0]
df['away_club_manager_name'] = games['away_club_manager_name'].mode()[0]

In [15]:
# attendance
df['attendance'] = request['attendance']

# referee
df['referee'] = request['referee']

In [16]:
df[['name', 'sub_type', 'type', 'country_id', 'is_major_national_league']] = competitions[competitions['competition_code'] == request['competition_code']][['name', 'sub_type', 'type', 'country_id', 'is_major_national_league']].iloc[0]

C:\Users\ACER\AppData\Local\Temp\ipykernel_5496\1960889964.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[['name', 'sub_type', 'type', 'country_id', 'is_major_national_league']] = competitions[competitions['competition_code'] == request['competition_code']][['name', 'sub_type', 'type', 'country_id', 'is_major_national_league']].iloc[0]


In [17]:
df[[
    'squad_size_home',
    'average_age_home', 'foreigners_number_home',
    'foreigners_percentage_home', 'national_team_players_home',
    'stadium_seats_home', 'last_season_home', 'total_market_value_home',
    'total_market_value_max_home', 'total_is_win_home',
]] = clubs[clubs['club_id'] == df['home_club_id'].values[0]][[
    'squad_size',
    'average_age', 'foreigners_number', 'foreigners_percentage',
    'national_team_players', 'stadium_seats',
    'last_season',
    'total_market_value', 'total_market_value_max', 'total_is_win'
]].values

In [18]:
df[[
    'squad_size_away',
    'average_age_away', 'foreigners_number_away',
    'foreigners_percentage_away', 'national_team_players_away',
    'stadium_seats_away', 'last_season_away', 'total_market_value_away',
    'total_market_value_max_away', 'total_is_win_away',
]] = clubs[clubs['club_id'] == df['away_club_id'].values[0]][[
    'squad_size',
    'average_age', 'foreigners_number', 'foreigners_percentage',
    'national_team_players', 'stadium_seats',
    'last_season',
    'total_market_value', 'total_market_value_max', 'total_is_win'
]].values

In [19]:
df['year'] = pd.to_datetime(request['date']).year
df['month'] = pd.to_datetime(request['date']).month
df['day'] = pd.to_datetime(request['date']).day
df['dayofweek'] = pd.to_datetime(request['date']).dayofweek

In [20]:
formation_code_map = joblib.load('./transformers/formation_code_map.joblib')

df['home_club_formation_code'] = formation_code_map[request['home_club_formation']]
df['away_club_formation_code'] = formation_code_map[request['away_club_formation']]

In [21]:
def parse_transfer_value(value):
    if pd.isnull(value):
        return None
    value = value.replace('€', '').replace('+', '').replace(' ', '')
    sign = -1 if '-' in value else 1
    value = value.replace('-', '')
    if value.endswith('m'):
        num = float(value[:-1]) * 1_000_000
    elif value.endswith('k'):
        num = float(value[:-1]) * 1_000
    else:
        num = float(value)
    return sign * num

# Приклад для DataFrame:
df['net_transfer_record_home_num'] = clubs[clubs['club_id'] == df['home_club_id'].values[0]]['net_transfer_record'].apply(parse_transfer_value).iloc[0]
df['net_transfer_record_away_num'] = clubs[clubs['club_id'] == df['away_club_id'].values[0]]['net_transfer_record'].apply(parse_transfer_value).iloc[0]

In [22]:
df['stadium_code'] = request['stadium']

In [23]:
df['stadium_name_home_code'] = clubs[clubs['club_id'] == df['home_club_id'].values[0]]['stadium_name'].iloc[0]
df['stadium_name_away_code'] = clubs[clubs['club_id'] == df['away_club_id'].values[0]]['stadium_name'].iloc[0]

In [24]:
df

,competition_id,season,round,home_club_id,away_club_id,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,attendance,referee,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_seats_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_seats_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_num,net_transfer_record_away_num,stadium_code,stadium_name_home_code,stadium_name_away_code
0,FRCH,2025,First Round,1421,347,3,3,Diego Simeone,Diego Simeone,21684,Eric Wattellier,trophee-des-champions,domestic_super_cup,other,50,False,25.0,24.9,19.0,76.0,8.0,20519.0,2024.0,146625000.0,259050000.0,119.0,26.0,25.0,19.0,73.1,7.0,28786.0,2023.0,68450000.0,237125000.0,58.0,2025,5,29,3,0,0,39350000.0,21900000.0,Stade Auguste-Delaune,Stade Auguste-Delaune,Stade Saint-Symphorien


In [25]:
df.to_csv('./example/formulated_request.csv', index=False)

# Обробка даних

In [26]:
stadium_code_map = joblib.load('./transformers/stadium_code_map.joblib')

df['stadium_code'] = df['stadium_code'].astype(str).map(stadium_code_map)
df['stadium_name_home_code'] = df['stadium_name_home_code'].astype(str).map(stadium_code_map)
df['stadium_name_away_code'] = df['stadium_name_away_code'].astype(str).map(stadium_code_map)

In [27]:
from numpy import dtype

factorize_maps = joblib.load('../res/transformers/label_encoding_maps.joblib')

def factorize_value(col, value):
    try:
        return factorize_maps[col].index(value)
    except ValueError:
        return -1

for col in [c for c in df.columns if df[c].dtype == dtype('O')]:
    df[col] = df[col].apply(lambda x: factorize_value(col, x))

### Нормалізація

In [28]:
scaler = joblib.load('./transformers/scaler.joblib')

cols_for_norm = df.drop('is_major_national_league', axis=1).columns
df[cols_for_norm] = scaler.transform(df[cols_for_norm])

In [30]:
df['is_major_national_league'] = df['is_major_national_league'].astype(float)

In [31]:
df

,competition_id,season,round,home_club_id,away_club_id,home_club_position,away_club_position,home_club_manager_name,away_club_manager_name,attendance,referee,name,sub_type,type,country_id,is_major_national_league,squad_size_home,average_age_home,foreigners_number_home,foreigners_percentage_home,national_team_players_home,stadium_seats_home,last_season_home,total_market_value_home,total_market_value_max_home,total_is_win_home,squad_size_away,average_age_away,foreigners_number_away,foreigners_percentage_away,national_team_players_away,stadium_seats_away,last_season_away,total_market_value_away,total_market_value_max_away,total_is_win_away,year,month,day,dayofweek,home_club_formation_code,away_club_formation_code,net_transfer_record_home_num,net_transfer_record_away_num,stadium_code,stadium_name_home_code,stadium_name_away_code
0,4.519153,1.908896,3.302116,-0.211531,-0.338251,-1.254505,-1.281171,-1.139683,-1.002943,0.09364,0.871778,4.634384,6.377698,5.439214,-0.922011,0.0,-0.116712,-0.545324,0.932925,1.312298,0.178833,-0.503274,0.4655,-0.211487,-0.392191,-0.489859,0.0404,-0.490333,0.933726,1.155821,0.002316,-0.074278,0.059587,-0.495792,-0.435796,-0.998675,1.767628,-0.433761,1.473068,-0.838094,-0.786416,-0.811003,1.191974,0.69951,-1.364131,-1.386368,0.178353


In [1]:
import torch
from torch import nn

In [2]:
class MyModel(nn.Module):
    def __init__(self, input, output):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(64, 8),
            nn.BatchNorm1d(8),
            nn.ReLU(),
            nn.Dropout(0.5),

            nn.Linear(8, output)
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [3]:
model_i = torch.load('./models/model_state_dict_epoch_11.pt', weights_only=False)

In [4]:
len(model_i['X_columns'])

47

In [5]:
X = joblib.load('./transformers/X_columns.joblib')
len(X)

NameError: name 'joblib' is not defined

In [6]:
model = MyModel(len(model_i['X_columns']), len(model_i['y_columns']))

In [7]:
model.load_state_dict(model_i['state_model'])

<All keys matched successfully>

In [9]:
import pandas as pd
df = pd.read_csv('./example/encoded.csv')

In [10]:
df.dtypes

competition_id                  float64
season                          float64
round                           float64
home_club_id                    float64
away_club_id                    float64
home_club_position              float64
away_club_position              float64
home_club_manager_name          float64
away_club_manager_name          float64
attendance                      float64
referee                         float64
name                            float64
sub_type                        float64
type                            float64
country_id                      float64
is_major_national_league        float64
squad_size_home                 float64
average_age_home                float64
foreigners_number_home          float64
foreigners_percentage_home      float64
national_team_players_home      float64
stadium_seats_home              float64
last_season_home                float64
total_market_value_home         float64
total_market_value_max_home     float64


In [13]:
model.eval()
pred = model(torch.tensor(df.values, dtype=torch.float32))

In [14]:
pred

tensor([[0.2278, 0.1041]], grad_fn=<AddmmBackward0>)

In [16]:
torch.save(model, './models/model_11.pt')

In [18]:
model_11 = torch.load('./models/model_11.pt', weights_only=False)

In [19]:
pred = model_11(torch.tensor(df.values, dtype=torch.float32))
pred

tensor([[0.2278, 0.1041]], grad_fn=<AddmmBackward0>)